In [50]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import json
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /Users/billpark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/billpark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/billpark/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [52]:
# Function to load JSON data into a DataFrame
def load_json_to_df(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return pd.DataFrame.from_dict(data, orient='index')

# List of JSON files
json_files = ['HR_prep.json', 'HF_prep.json', 'MR_prep.json', 'MF_prep.json']
dfs = []

# Load each JSON file into a DataFrame and store them in a list
for json_file in json_files:
    df = load_json_to_df('data/' + json_file)
    dfs.append(df)

# Naming the DataFrames
HR_df, HF_df, MR_df, MF_df = dfs

In [53]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [54]:
HR = HR_df['preprocessed_text']
HF = HF_df['preprocessed_text']
MR = MR_df['preprocessed_text']
MF = MF_df['preprocessed_text']

In [55]:
df = pd.concat([HR, HF, MR, MF], ignore_index=True)

In [56]:
from sklearn.metrics import accuracy_score, classification_report

# Assigning labels for each comparison
labels_hr_vs_mf = [0] * len(HR) + [1] * len(MF)
labels_hf_vs_mf = [0] * len(HF) + [1] * len(MF)
labels_mr_vs_mf = [0] * len(MR) + [1] * len(MF)
labels_hf_vs_mr = [0] * len(HF) + [1] * len(MR)
labels_hr_vs_mr = [0] * len(HR) + [1] * len(MR)
labels_hr_vs_hf = [0] * len(HR) + [1] * len(HF)

In [57]:
# Splitting the data for each comparison
X_train_hr_vs_mf, X_test_hr_vs_mf, y_train_hr_vs_mf, y_test_hr_vs_mf = train_test_split(pd.concat([HR, MF], ignore_index=True), labels_hr_vs_mf, test_size=0.2, random_state=42)
X_train_hf_vs_mf, X_test_hf_vs_mf, y_train_hf_vs_mf, y_test_hf_vs_mf = train_test_split(pd.concat([HF, MF], ignore_index=True), labels_hf_vs_mf, test_size=0.2, random_state=42)
X_train_mr_vs_mf, X_test_mr_vs_mf, y_train_mr_vs_mf, y_test_mr_vs_mf = train_test_split(pd.concat([MR, MF], ignore_index=True), labels_mr_vs_mf, test_size=0.2, random_state=42)
X_train_hf_vs_mr, X_test_hf_vs_mr, y_train_hf_vs_mr, y_test_hf_vs_mr = train_test_split(pd.concat([HF, MR], ignore_index=True), labels_hf_vs_mr, test_size=0.2, random_state=42)
X_train_hr_vs_mr, X_test_hr_vs_mr, y_train_hr_vs_mr, y_test_hr_vs_mr = train_test_split(pd.concat([HR, MR], ignore_index=True), labels_hr_vs_mr, test_size=0.2, random_state=42)
X_train_hr_vs_hf, X_test_hr_vs_hf, y_train_hr_vs_hf, y_test_hr_vs_hf = train_test_split(pd.concat([HR, HF], ignore_index=True), labels_hr_vs_hf, test_size=0.2, random_state=42)


In [58]:

# Creating TF-IDF vectorizer for each comparison
tfidf_vectorizer_hr_vs_mf = TfidfVectorizer(max_features=5000)
X_train_hr_vs_mf_tfidf = tfidf_vectorizer_hr_vs_mf.fit_transform(X_train_hr_vs_mf)
X_test_hr_vs_mf_tfidf = tfidf_vectorizer_hr_vs_mf.transform(X_test_hr_vs_mf)

tfidf_vectorizer_hf_vs_mf = TfidfVectorizer(max_features=5000)
X_train_hf_vs_mf_tfidf = tfidf_vectorizer_hf_vs_mf.fit_transform(X_train_hf_vs_mf)
X_test_hf_vs_mf_tfidf = tfidf_vectorizer_hf_vs_mf.transform(X_test_hf_vs_mf)

tfidf_vectorizer_mr_vs_mf = TfidfVectorizer(max_features=5000)
X_train_mr_vs_mf_tfidf = tfidf_vectorizer_mr_vs_mf.fit_transform(X_train_mr_vs_mf)
X_test_mr_vs_mf_tfidf = tfidf_vectorizer_mr_vs_mf.transform(X_test_mr_vs_mf)

tfidf_vectorizer_hf_vs_mr = TfidfVectorizer(max_features=5000)
X_train_hf_vs_mr_tfidf = tfidf_vectorizer_hf_vs_mr.fit_transform(X_train_hf_vs_mr)
X_test_hf_vs_mr_tfidf = tfidf_vectorizer_hf_vs_mr.transform(X_test_hf_vs_mr)

tfidf_vectorizer_hr_vs_mr = TfidfVectorizer(max_features=5000)
X_train_hr_vs_mr_tfidf = tfidf_vectorizer_hr_vs_mr.fit_transform(X_train_hr_vs_mr)
X_test_hr_vs_mr_tfidf = tfidf_vectorizer_hr_vs_mr.transform(X_test_hr_vs_mr)

tfidf_vectorizer_hr_vs_hf = TfidfVectorizer(max_features=5000)
X_train_hr_vs_hf_tfidf = tfidf_vectorizer_hr_vs_hf.fit_transform(X_train_hr_vs_hf)
X_test_hr_vs_hf_tfidf = tfidf_vectorizer_hr_vs_hf.transform(X_test_hr_vs_hf)

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from xgboost import XGBClassifier
import pandas as pd

# Function to train and evaluate an XGBoost model
def train_and_evaluate_xgb(X_train, y_train, X_test, y_test):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print("Accuracy:", accuracy_score(y_test, predictions))

# Using the function to train and evaluate models for each comparison
print("HR vs MF:")
train_and_evaluate_xgb(X_train_hr_vs_mf_tfidf, y_train_hr_vs_mf, X_test_hr_vs_mf_tfidf, y_test_hr_vs_mf)
print("HF vs MF:")
train_and_evaluate_xgb(X_train_hf_vs_mf_tfidf, y_train_hf_vs_mf, X_test_hf_vs_mf_tfidf, y_test_hf_vs_mf)
print("MR vs MF:")
train_and_evaluate_xgb(X_train_mr_vs_mf_tfidf, y_train_mr_vs_mf, X_test_mr_vs_mf_tfidf, y_test_mr_vs_mf)
print("HF vs MR:")
train_and_evaluate_xgb(X_train_hf_vs_mr_tfidf, y_train_hf_vs_mr, X_test_hf_vs_mr_tfidf, y_test_hf_vs_mr)
print("HR vs MR:")
train_and_evaluate_xgb(X_train_hr_vs_mr_tfidf, y_train_hr_vs_mr, X_test_hr_vs_mr_tfidf, y_test_hr_vs_mr)
print("HR vs HF:")
train_and_evaluate_xgb(X_train_hr_vs_hf_tfidf, y_train_hr_vs_hf, X_test_hr_vs_hf_tfidf, y_test_hr_vs_hf)


HR vs MF:
Accuracy: 0.9696601941747572
HF vs MF:
Accuracy: 0.9608323133414932
MR vs MF:
Accuracy: 0.9400363416111448
HF vs MR:
Accuracy: 0.9170199878861296
HR vs MR:
Accuracy: 0.8774774774774775
HR vs HF:
Accuracy: 0.7779126213592233


Word2Vec attemp from below

In [ ]:
from gensim.models import Word2Vec

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

def text_to_wordlist(text):
    return word_tokenize(text)

def build_word2vec_model(data_frames, text_extractor):
    sentences = []
    for df in data_frames:
        if isinstance(df, pd.DataFrame) and not df.empty:
            for _, row in df.iterrows():
                extracted_text = text_extractor(row)
                tokenized_text = text_to_wordlist(extracted_text)
                sentences.append(tokenized_text)
        else:
            print("Error: Provided data is not a DataFrame or is empty.")

    if sentences:
        model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
        return model
    else:
        raise ValueError("No sentences were extracted for Word2Vec training. Check your data and text extraction logic.")

def vectorize_with_word2vec(df, word2vec_model, text_extractor):
    def average_vector(row):
        text = text_extractor(row)
        words = text_to_wordlist(text)
        word_vectors = [word2vec_model.wv[word] for word in words if word in word2vec_model.wv]
        if len(word_vectors) == 0:
            return np.zeros(word2vec_model.vector_size)
        else:
            return np.mean(word_vectors, axis=0)
    return np.vstack(df.apply(average_vector, axis=1))

def prepare_and_vectorize_data(df1, df2, word2vec_model, text_extractor):
    labels = [0] * len(df1) + [1] * len(df2)
    X = pd.concat([df1, df2], ignore_index=True)
    X_vectors = vectorize_with_word2vec(X, word2vec_model, text_extractor)
    X_train, X_test, y_train, y_test = train_test_split(X_vectors, labels, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

def train_and_evaluate_xgb(X_train, y_train, X_test, y_test, comparison_name):
    model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"XGBoost Accuracy for {comparison_name}: {accuracy:.6f}")

# Assuming HR_df, HF_df, MR_df, MF_df are DataFrame objects and are loaded correctly
word2vec_model = build_word2vec_model([HR_df, HF_df, MR_df, MF_df], lambda row: extract_text(row, 'preprocessed_text', 'text'))

# Define pairs for comparison
pairs = [
    (HR_df, MF_df, "Human Real vs AI Fake"),
    (HF_df, MF_df, "Human Fake vs AI Fake"),
    (MR_df, MF_df, "AI Real vs AI Fake"),
    (HF_df, MR_df, "Human Fake vs AI Real"),
    (HR_df, MR_df, "Human Real vs AI Real"),
    (HR_df, HF_df, "Human Real vs Human Fake")
]

for df1, df2, name in pairs:
    X_train, X_test, y_train, y_test = prepare_and_vectorize_data(df1, df2, word2vec_model, lambda row: extract_text(row, 'preprocessed_text', 'text'))
    train_and_evaluate_xgb(X_train, y_train, X_test, y_test, name)


[nltk_data] Downloading package punkt to /Users/billpark/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


XGBoost Accuracy for Human Real vs AI Fake: 0.956311
XGBoost Accuracy for Human Fake vs AI Fake: 0.925337
XGBoost Accuracy for AI Real vs AI Fake: 0.910357
XGBoost Accuracy for Human Fake vs AI Real: 0.898243
XGBoost Accuracy for Human Real vs AI Real: 0.807808
XGBoost Accuracy for Human Real vs Human Fake: 0.788835
